# アンサンブル学習（Blending）と提出

In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import keras
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

Using TensorFlow backend.


別に作成していた各モデルのロード

In [7]:
with open('lgbs_7cv.sav', mode='rb') as f:  
    lgbs = pickle.load(f) 
with open('opts_7.sav', mode='rb') as f:  
    opts = pickle.load(f) 
with open('xgbs_7cv.sav', mode='rb') as f:  
    xgbs = pickle.load(f) 
with open('svms_7cv.sav', mode='rb') as f:  
    svms = pickle.load(f) 
with open('ridges_7cv.sav', mode='rb') as f:
    ridges = pickle.load(f) 
with open('forests_7.sav', mode='rb') as f:
    forests = pickle.load(f) 

[15:00:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:00:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:00:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:00:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:00:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [3]:
neural = tf.keras.models.load_model('neural_7.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
# trainデータのロード
df_7 = pd.read_csv('data_7.csv')
display(df_7.head())

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2,IC_01,IC_07,IC_05,IC_16,...,CBN(13)_vs_DMN(94),CBN(18)_vs_DMN(94),CBN(4)_vs_DMN(94),CBN(7)_vs_DMN(94),CBN(18)_vs_CBN(13),CBN(4)_vs_CBN(13),CBN(7)_vs_CBN(13),CBN(4)_vs_CBN(18),CBN(7)_vs_CBN(18),CBN(7)_vs_CBN(4)
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998,0.006070,0.014466,0.004136,0.000658,...,-0.203459,0.630524,0.227728,0.407087,0.450352,-0.074469,0.151244,-0.105504,0.626158,0.352528
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361,0.009087,0.009291,0.007049,-0.002076,...,-0.306443,-0.047546,0.260530,-0.207901,0.595487,0.809261,0.648386,0.052190,0.772152,0.271778
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641,0.004675,0.000957,0.006154,-0.000429,...,-0.276111,0.393040,0.370413,0.042790,0.437297,0.560578,0.440411,0.032559,0.923573,0.112833
3,10005,66.532630,51.847306,60.052535,52.108977,69.993075,-0.000398,0.006878,0.009051,0.000369,...,-0.248033,0.522701,0.107870,0.605457,0.660222,0.711500,0.321404,0.077306,1.063450,0.124246
4,10007,38.617381,49.197021,65.674285,40.151376,34.096421,0.005192,0.010585,0.012160,-0.000920,...,-0.178258,0.387695,0.166441,0.176302,0.504423,0.821638,0.517638,0.372172,0.787271,0.328486


In [5]:
# testデータのロード
df_test7 = pd.read_csv('df_test7.csv')
display(df_test7.head())

,Id,IC_01,IC_07,IC_05,IC_16,IC_26,IC_06,IC_10,IC_09,IC_18,...,CBN(13)_vs_DMN(94),CBN(18)_vs_DMN(94),CBN(4)_vs_DMN(94),CBN(7)_vs_DMN(94),CBN(18)_vs_CBN(13),CBN(4)_vs_CBN(13),CBN(7)_vs_CBN(13),CBN(4)_vs_CBN(18),CBN(7)_vs_CBN(18),CBN(7)_vs_CBN(4)
0,10003,0.008151,0.014684,0.010444,-0.005293,-0.002913,0.015042,0.017745,0.003930,-0.008021,...,-0.197880,0.113676,-0.026490,0.200988,0.563162,0.485028,0.823505,0.519705,0.902823,0.415444
1,10006,0.000334,0.005311,0.010053,0.006920,-0.000065,0.015310,0.016543,0.004794,0.003982,...,-0.060595,0.271157,0.310912,0.344462,0.398932,0.559192,0.390569,0.327991,0.649678,0.420190
2,10010,0.007103,0.006144,0.009770,-0.002884,-0.001346,0.015651,0.011613,-0.003291,0.013423,...,-0.348429,0.348177,0.500079,0.282466,0.721727,0.321671,0.718345,0.215431,1.137625,0.190011
3,10011,0.004362,0.010240,0.010167,0.004492,-0.001623,0.017381,0.014680,0.007453,0.008786,...,-0.139529,0.577357,0.102322,0.249017,0.431676,1.069165,0.606264,0.396510,0.820862,0.425553
4,10012,-0.007521,-0.003918,0.008434,-0.001145,0.002017,0.015065,0.019616,0.004140,-0.003744,...,-0.029925,0.197446,0.309573,0.050748,0.545433,0.746623,0.671737,0.389646,0.870958,0.331737


trainデータ、testデータのそれぞれにつき各モデルの予測値を出力

In [10]:
# lightGBM
lgb_train = np.empty([X_train.shape[0], 5])
lgb_preds = np.empty([X_test.shape[0], 5])
for i in range(5):
    lgb_train[:, i] = lgbs[i].predict(X_train)
    lgb_preds[:, i] = lgbs[i].predict(X_test)

In [11]:
# Optuna
opt_train = np.empty([X_train.shape[0], 5])
opt_preds = np.empty([X_test.shape[0], 5])
for i in range(5):
    opt_train[:, i] = opts[i].predict(X_train)
    opt_preds[:, i] = opts[i].predict(X_test)

In [12]:
# XGBoost
xgb_train = np.empty([X_train.shape[0], 5])
xgb_preds = np.empty([X_test.shape[0], 5])
for i in range(5):
    xgb_train[:, i] = xgbs[i].predict(X_train)
    xgb_preds[:, i] = xgbs[i].predict(X_test)

In [13]:
# SVR
svm_train = np.empty([X_train.shape[0], 5])
svm_preds = np.empty([X_test.shape[0], 5])
for i in range(5):
    svm_train[:, i] = svms[i].predict(X_train_std)
    svm_preds[:, i] = svms[i].predict(X_test_std)
svm_train = sc_y.inverse_transform(svm_train)
svm_preds = sc_y.inverse_transform(svm_preds)

In [14]:
# Ridge
ridge_train = np.empty([X_train.shape[0], 5])
ridge_preds = np.empty([X_test.shape[0], 5])
for i in range(5):
    ridge_train[:, i] = ridges[i].predict(X_train_std)
    ridge_preds[:, i] = ridges[i].predict(X_test_std)
ridge_train = sc_y.inverse_transform(ridge_train)
ridge_preds = sc_y.inverse_transform(ridge_preds)

In [22]:
# NeuralNetwork
nn_train = neural.predict(X_train_tsf)
nn_preds = neural.predict(X_test_tsf)
nn_train = sc_y.inverse_transform(nn_train)
nn_preds = sc_y.inverse_transform(nn_preds)

In [17]:
# Random Forest
forest_train = np.empty([X_train.shape[0], 5])
forest_preds = np.empty([X_test.shape[0], 5])
for i in range(5):
    forest_train[:, i] = forests[i].predict(X_train_std)
    forest_preds[:, i] = forests[i].predict(X_test_std)
forest_train = sc_y.inverse_transform(forest_train)
forest_preds = sc_y.inverse_transform(forest_preds)

In [30]:
# 予測値一覧
for i in range(5):
    trains[i].iloc[:, 0] = lgb_train[:, i]
    trains[i].iloc[:, 1] = opt_train[:, i]
    trains[i].iloc[:, 2] = xgb_train[:, i]
    trains[i].iloc[:, 3] = svm_train[:, i]
    trains[i].iloc[:, 4] = ridge_train[:, i]
    trains[i].iloc[:, 5] = nn_train[:, i]
    trains[i].iloc[:, 6] = forest_train[:, i]

trains[0]

,lgm,opt,xgb,svm,ridge,neural,forest
0,63.290071,63.921451,51.157475,60.458973,56.943540,59.771938,49.362789
1,56.688701,57.013727,49.930653,58.540662,61.704871,57.960415,50.934697
2,60.164456,61.753425,53.592022,59.415679,58.746751,59.620605,50.212925
3,61.111772,65.205509,52.720730,65.377912,64.595355,61.031750,50.256785
4,40.023037,42.418136,49.964783,40.362186,42.394858,39.872894,50.836490
...,...,...,...,...,...,...,...
5872,40.603760,26.259666,53.357674,40.250103,44.572403,29.051067,49.049924
5873,50.872782,53.563173,51.077812,49.572319,60.343316,55.568245,50.208376
5874,46.992199,46.967169,51.748642,44.246553,38.291022,50.535759,50.079592
5875,55.937719,58.988872,49.491840,57.100983,54.959444,64.700165,50.005993


最も単純なBlendingでは、各モデルの平均をとる。

In [110]:
pred = (lgb_preds + xgb_preds + svm_preds + ridge_preds + n_preds)/5
pred

array([[62.19803497, 51.24563913, 56.9784374 , 50.93515138, 57.7781543 ],
       [60.3108791 , 53.25306399, 58.18840562, 49.86629149, 49.6286171 ],
       [41.50346622, 46.92977383, 58.2761202 , 46.2265802 , 52.10342782],
       ...,
       [41.80945641, 48.08672223, 58.90227341, 45.87878972, 50.48727307],
       [41.94637828, 49.56406937, 52.90169533, 45.98624048, 54.07750027],
       [43.89592246, 48.77936598, 61.67000344, 49.55381467, 53.1330257 ]])

Blendigでは単純平均の他、各モデルの加重平均をとる。  
この重みの最適化に、Scipyのoptimizeライブラリを使用する。  
真値のわかっているtrainデータをもとに、「各モデルの重みをいくつに設定すれば、設定したscoreを最小にできたか」で最適化される。

In [31]:
# 予測値と最小化したいscoreの定義
def score_func(weights):
    final_prediction = 0
    for weight, prediction in zip(weights, predictions):
        final_prediction += weight*prediction
    return np.sum(np.abs(y_values - final_prediction))/np.sum(final_prediction)

In [36]:
from scipy.optimize import minimize

for i in range(5):
    predictions = []
    for j in range(7):
        predictions.append(np.array(trains[i].iloc[:, j]))
    lls = []
    wghts = []
    y_values = y[:,i]
    for k in range(100):
        starting_values = np.random.uniform(size=7)
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        bounds = [(0,1)]*len(predictions)
        res = minimize(score_func, starting_values, constraints=cons,
                       bounds = bounds, method='SLSQP')

        lls.append(res['fun'])
        wghts.append(res['x'])
    
    bestSC = np.min(lls)
    bestWght = wghts[np.argmin(lls)]
    print(i)
    print('\n Ensemble Score: {:.5f}'.format(bestSC))
    print('\n Best Weights: {}'.format(bestWght))

0

 Ensemble Score: 0.08234

 Best Weights: [0.00000000e+00 1.38248130e-01 3.41227548e-20 3.93475835e-01
 1.22598490e-18 4.68276035e-01 1.70765278e-19]
1

 Ensemble Score: 0.11791

 Best Weights: [2.29204992e-17 7.82675432e-01 0.00000000e+00 2.79183204e-18
 2.17324568e-01 2.67469155e-18 0.00000000e+00]
2

 Ensemble Score: 0.12639

 Best Weights: [0.00000000e+00 5.20417043e-17 0.00000000e+00 6.05502797e-01
 3.94497203e-01 3.82513014e-19 0.00000000e+00]
3

 Ensemble Score: 0.15453

 Best Weights: [1.31758960e-17 2.00099697e-16 1.18447174e-19 5.46738616e-20
 3.61086275e-01 6.38913725e-01 5.11789535e-17]
4

 Ensemble Score: 0.15446

 Best Weights: [0.00000000e+00 5.36092095e-01 0.00000000e+00 1.47494774e-17
 4.63907905e-01 6.34972153e-18 0.00000000e+00]


この結果からもSVMやRidgeの重みが相対的に大きくなっており、今回の予測には決定木系のモデルがあまり有効でなかったことが示唆される。

In [38]:
age_w = np.array([0.00000000e+00, 1.38248130e-01, 3.41227548e-20, 3.93475835e-01,
 1.22598490e-18, 4.68276035e-01, 1.70765278e-19])

d1v1_w = np.array([2.29204992e-17, 7.82675432e-01, 0.00000000e+00, 2.79183204e-18,
 2.17324568e-01, 2.67469155e-18, 0.00000000e+00])

d1v2_w = np.array([0.00000000e+00, 5.20417043e-17, 0.00000000e+00, 6.05502797e-01,
 3.94497203e-01, 3.82513014e-19, 0.00000000e+00])

d2v1_w = np.array([1.31758960e-17, 2.00099697e-16, 1.18447174e-19, 5.46738616e-20,
 3.61086275e-01, 6.38913725e-01, 5.11789535e-17])

d2v2_w = np.array([0.00000000e+00, 5.36092095e-01, 0.00000000e+00, 1.47494774e-17,
 4.63907905e-01, 6.34972153e-18, 0.00000000e+00])

上記のままではあまりに重みに偏りがあるため、これと単純平均との折衷も別に作成する。

In [41]:
age_w = np.array([0.00000000e+00, 1.38248130e-01, 3.41227548e-20, 3.93475835e-01,
 1.22598490e-18, 4.68276035e-01, 1.70765278e-19]) + np.ones(7)
age_w = age_w / np.sum(age_w)
print(age_w)

d1v1_w = np.array([2.29204992e-17, 7.82675432e-01, 0.00000000e+00, 2.79183204e-18,
 2.17324568e-01, 2.67469155e-18, 0.00000000e+00]) + np.ones(7)
d1v1_w = d1v1_w / np.sum(d1v1_w)
print(d1v1_w)

d1v2_w = np.array([0.00000000e+00, 5.20417043e-17, 0.00000000e+00, 6.05502797e-01,
 3.94497203e-01, 3.82513014e-19, 0.00000000e+00]) + np.ones(7)
d1v2_w = d1v2_w / np.sum(d1v2_w)
print(d1v2_w)

d2v1_w = np.array([1.31758960e-17, 2.00099697e-16, 1.18447174e-19, 5.46738616e-20,
 3.61086275e-01, 6.38913725e-01, 5.11789535e-17]) + np.ones(7)
d2v1_w = d2v1_w / np.sum(d2v1_w)
print(d2v1_w)

d2v2_w = np.array([0.00000000e+00, 5.36092095e-01, 0.00000000e+00, 1.47494774e-17,
 4.63907905e-01, 6.34972153e-18, 0.00000000e+00]) + np.ones(7)
d2v2_w = d2v2_w / np.sum(d2v2_w)
print(d2v2_w)

[0.125      0.14228102 0.125      0.17418448 0.125      0.1835345
 0.125     ]
[0.125      0.22283443 0.125      0.125      0.15216557 0.125
 0.125     ]
[0.125      0.125      0.125      0.20068785 0.17431215 0.125
 0.125     ]
[0.125      0.125      0.125      0.125      0.17013578 0.20486422
 0.125     ]
[0.125      0.19201151 0.125      0.125      0.18298849 0.125
 0.125     ]


上記3通りの方法で重み（``weights``）を決定し、testデータから提出用のcsvファイルを作成する。

In [42]:
ID_test = df_test7['Id']
preds = []
weights = [age_w, d1v1_w, d1v2_w, d2v1_w, d2v2_w]

for i in range(5):
    premat = np.vstack([lgb_preds[:,i], 
                        opt_preds[:,i], 
                        xgb_preds[:,i], 
                        svm_preds[:,i], 
                        ridge_preds[:,i],
                        nn_preds[:,i],
                        forest_preds[:,i]])
    weight = (weights[i][:,None]).T
    prevec = weight @ premat
    preds.append(prevec.flatten())

ID_age = ID_test.astype('str') + np.array(['_age'], dtype=object)
df_age = pd.DataFrame({'Id': ID_age, 'Predicted': preds[0]})
ID_d1v1 = ID_test.astype('str') + np.array(['_domain1_var1'], dtype=object)
df_d1v1 = pd.DataFrame({'Id': ID_d1v1, 'Predicted': preds[1]})
ID_d1v2 = ID_test.astype('str') + np.array(['_domain1_var2'], dtype=object)
df_d1v2 = pd.DataFrame({'Id': ID_d1v2, 'Predicted': preds[2]})
ID_d2v1 = ID_test.astype('str') + np.array(['_domain2_var1'], dtype=object)
df_d2v1 = pd.DataFrame({'Id': ID_d2v1, 'Predicted': preds[3]})
ID_d2v2 = ID_test.astype('str') + np.array(['_domain2_var2'], dtype=object)
df_d2v2 = pd.DataFrame({'Id': ID_d2v2, 'Predicted': preds[4]})

df_sub = pd.concat([df_age, df_d1v1, df_d1v2, df_d2v1, df_d2v2], axis=0)
df_sub = df_sub.sort_values('Id')
display(df_sub)
df_sub.to_csv('submission11', index=False, header=True)

,Id,Predicted
0,10003_age,57.520608
0,10003_domain1_var1,50.767184
0,10003_domain1_var2,57.784263
0,10003_domain2_var1,50.461669
0,10003_domain2_var2,57.655053
...,...,...
5876,21753_age,44.585045
5876,21753_domain1_var1,48.687010
5876,21753_domain1_var2,61.990406
5876,21753_domain2_var1,49.455184


これらを提出してみたところ、単純平均と最適化した重みを折衷したものがpubric leaderboardで最もスコアが良かった。（0.164）